In [1]:

import os
import requests
from glob import glob
from requests.utils import requote_uri
from osgeo import gdal, ogr, osr
import tempfile
from pathlib import Path
import pandas as pd
import json
from pyproj import Transformer
from calendar import monthrange


In [2]:
geojson_fname = '/workspace/WorldPeatland/sites/MerBleue.geojson'

In [3]:

def get_polygon(fname):
    """
    Get POLYGON string from GeoJSON coordinates
    """
    with open(fname) as f:
        geojson = json.load(f)

    polygon = ""
    for coordinate in geojson['features'][0]['geometry']['coordinates'][0]:
        _coordinate = " ".join(str(x) for x in coordinate)
        polygon = f"{polygon} {_coordinate}, "

    polygon = polygon[0:-2]

    return polygon


In [4]:
polygon = get_polygon(geojson_fname)

In [5]:
url_start = (f"https://datahub.creodias.eu/odata/v1/Products?$filter=")

In [6]:
url_end = (f"(Online eq true) and "
           f"(OData.CSC.Intersects(Footprint=geography%27SRID=4326;POLYGON%20(("
           f"{polygon}"
           f"))%27)) and "
           f"(((((Collection/Name eq %27SENTINEL-1%27) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27platformSerialIdentifier%27 and i0/Value eq %27A%27)) or "
           "(Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27platformSerialIdentifier%27 and i0/Value eq %27B%27)))) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27productType%27 and%20i0/Value eq %27GRD-COG%27)))) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27operationalMode%27 and 0i0/Value eq %27IW%27)))) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27orbitDirection%27 and i0/Value eq %27ASCENDING%27)) or "
           "(Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27orbitDirection%27 and i0/Value eq %27DESCENDING%27)))) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27polarisationChannels%27 and i0/Value eq %27VV%26VH%27))))))))"
           f")&$expand=Attributes&$expand=Assets&$orderby=ContentDate/Start asc&$top=20")

In [7]:
for year in range(2017, 2024+1):
    for month in range(1, 12+1):
        start_date = f'{year}-{month:02}-01T00:00:00.000Z'
        end_day = monthrange(year, month)[1]
        end_date = f'{year}-{month:02}-{end_day:02}T23:59:59.999Z'

        url = (f"{url_start}"   
               f"((ContentDate/Start ge {start_date} and ContentDate/Start le {end_date}) and "
               f"{url_end}")
        
        url_encoded = requote_uri(url)
        url_encoded_cleared = url_encoded.replace('%0A', '')
        
        response = requests.get(url_encoded_cleared)
        response = response.json()

        print(url_encoded_cleared)
        break
        
    break


https://datahub.creodias.eu/odata/v1/Products?$filter=((ContentDate/Start%20ge%202017-01-01T00:00:00.000Z%20and%20ContentDate/Start%20le%202017-01-31T23:59:59.999Z)%20and%20(Online%20eq%20true)%20and%20(OData.CSC.Intersects(Footprint=geography%27SRID=4326;POLYGON%20((%20-75.58009416302863%2045.423671289918474,%20%20-75.58009416302863%2045.37776159126091,%20%20-75.4396002179461%2045.37776159126091,%20%20-75.4396002179461%2045.423671289918474,%20%20-75.58009416302863%2045.423671289918474))%27))%20and%20(((((Collection/Name%20eq%20%27SENTINEL-1%27)%20and%20(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name%20eq%20%27platformSerialIdentifier%27%20and%20i0/Value%20eq%20%27A%27))%20or%20(Attributes/OData.CSC.StringAttribute/any(i0:i0/Name%20eq%20%27platformSerialIdentifier%27%20and%20i0/Value%20eq%20%27B%27))))%20and%20(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name%20eq%20%27productType%27%20and%20i0/Value%20eq%20%27GRD-COG%27))))%20and%20(((Attributes/OData.CSC.StringAttribute/a

In [8]:
cloud_cover_le = 30

In [9]:
url_start = (f"https://datahub.creodias.eu/odata/v1/Products?$filter="
             f"((Attributes/OData.CSC.DoubleAttribute/any(i0:i0/Name eq %27cloudCover%27 and i0/Value le {cloud_cover_le})) and ")

url_end = (f"(Online eq true) and "
           f"(OData.CSC.Intersects(Footprint=geography%27SRID=4326;POLYGON%20(("
           f"{polygon}"
           f"))%27)) and "
           f"(((((Collection/Name eq %27SENTINEL-2%27) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27productType%27 and i0/Value eq %27S2MSI2A%27)))) and "
           f"(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27processorVersion%27 and i0/Value eq %2705.00%27)) or (Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq %27processorVersion%27 and i0/Value eq %2705.09%27))))))))"
           f")&$expand=Attributes&$expand=Assets&$orderby=ContentDate/Start asc&$top=200")


In [10]:
for year in range(2017, 2024+1):
    for month in range(1, 12+1):
        start_date = f'{year}-{month:02}-01T00:00:00.000Z'
        end_day = monthrange(year, month)[1]
        end_date = f'{year}-{month:02}-{end_day:02}T23:59:59.999Z'

        url = (f"{url_start}"
               f"(ContentDate/Start ge {start_date} and ContentDate/Start le {end_date}) and "
               f"{url_end}")

        # Encode URL
        url_encoded = requote_uri(url)

        # Remove unnecessasary characters from encoded URL
        url_encoded_cleared = url_encoded.replace('%0A', '')

        # Obtain and print the response
        response = requests.get(url_encoded_cleared)
        response = response.json()
        
        print(response)

        break

{'@odata.context': '$metadata#Products(Attributes())(Assets())', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': '90e4ad75-9af1-409f-9856-f09237784baa', 'Name': 'S2A_MSIL2A_20170106T155631_N0500_R054_T18TVR_20230920T062905.SAFE', 'ContentType': 'application/octet-stream', 'ContentLength': 930657445, 'OriginDate': '2024-04-20T07:41:15.110Z', 'PublicationDate': '2024-04-20T07:49:21.337Z', 'ModificationDate': '2024-04-30T23:17:47.205Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-2/MSI/L2A_N0500/2017/01/06/S2A_MSIL2A_20170106T155631_N0500_R054_T18TVR_20230920T062905.SAFE', 'Checksum': [{'Value': 'c0f6e5b2b54e44652abfbb3516afaedc', 'Algorithm': 'MD5', 'ChecksumDate': '2024-04-20T07:49:34.098670Z'}, {'Value': 'fba49e8c2318c36fdc3a0307a5dbea6a74e71f2134e8e3e2a4cd2854e8e7323d', 'Algorithm': 'BLAKE3', 'ChecksumDate': '2024-04-20T07:49:35.911170Z'}], 'ContentDate': {'Start': '2017-01-06T15:56:31.026Z', 'End': '2017-01-06T15:56:31.026Z'}, 'Footprint': 

{'@odata.context': '$metadata#Products(Attributes())(Assets())', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': 'ddf2470b-7537-4b72-a67e-7d189d0ae311', 'Name': 'S2A_MSIL2A_20180124T160541_N0500_R097_T18TVR_20230915T133805.SAFE', 'ContentType': 'application/octet-stream', 'ContentLength': 1232006092, 'OriginDate': '2024-03-14T23:45:41.662Z', 'PublicationDate': '2024-03-15T00:11:12.735Z', 'ModificationDate': '2024-04-26T21:53:21.729Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/24/S2A_MSIL2A_20180124T160541_N0500_R097_T18TVR_20230915T133805.SAFE', 'Checksum': [{'Value': '9741c908a38a8b5f71cfa67b22fd2beb', 'Algorithm': 'MD5', 'ChecksumDate': '2024-03-15T00:11:48.958561Z'}, {'Value': '5960912b06aefeddc05b8357328acc3ce7ffff2b363d32044b2dcd66dde74e6e', 'Algorithm': 'BLAKE3', 'ChecksumDate': '2024-03-15T00:11:51.529074Z'}], 'ContentDate': {'Start': '2018-01-24T16:05:41.026Z', 'End': '2018-01-24T16:05:41.026Z'}, 'Footprint':

{'@odata.context': '$metadata#Products(Attributes())(Assets())', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': '2d739b4a-1601-4c6d-9717-05bc7fdae199', 'Name': 'S2A_MSIL2A_20210108T160641_N0500_R097_T18TVR_20230602T163101.SAFE', 'ContentType': 'application/octet-stream', 'ContentLength': 1277100625, 'OriginDate': '2023-07-07T05:57:18.639Z', 'PublicationDate': '2023-07-07T23:24:55.482Z', 'ModificationDate': '2024-05-07T16:04:47.756Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-2/MSI/L2A_N0500/2021/01/08/S2A_MSIL2A_20210108T160641_N0500_R097_T18TVR_20230602T163101.SAFE', 'Checksum': [{'Value': '761c04733e94266ad37c3df90f8d6f2a', 'Algorithm': 'MD5', 'ChecksumDate': '2023-07-07T23:25:13.809026Z'}, {'Value': 'c10ed242e9133d8cf15bbfef386bdfb216c8a724dad1f0acd6ed77bc4dc4efa9', 'Algorithm': 'BLAKE3', 'ChecksumDate': '2023-07-07T23:25:16.578989Z'}], 'ContentDate': {'Start': '2021-01-08T16:06:41.024Z', 'End': '2021-01-08T16:06:41.024Z'}, 'Footprint':